Importamos las librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.6f' % x)

# Conectividad por Localidad

El primer análisis que realizaremos es en función a la conectividad por localidad 

Cargamos los datos

In [ ]:
df_conectividad_por_localidad = pd.read_csv('_src/data/Lis_loc_conectividad_a_internet_2750221689173466318.csv', encoding='UTF-8')

In [ ]:
df_conectividad_por_localidad

Reemplazamos los carácteres '--' que representan sin conectividad a 'NO' para hacerlo mas explicito

In [ ]:
df_conectividad_por_localidad = df_conectividad_por_localidad.replace('--','NO')

Por motivos practicos crearemos una columna que explique si la localidad presenta o no conectividad a internet, por cualquier medio

In [ ]:
sin_conectividad = (df_conectividad_por_localidad['ADSL'] == 'NO') & (df_conectividad_por_localidad['CABLEMODEM'] == 'NO') & (df_conectividad_por_localidad['DIALUP'] == 'NO') & (df_conectividad_por_localidad['FIBRAOPTICA'] == 'NO') & (df_conectividad_por_localidad['4G'] == 'NO') & (df_conectividad_por_localidad['3G'] == 'NO') & (df_conectividad_por_localidad['TELEFONIAFIJA'] == 'NO') & (df_conectividad_por_localidad['WIRELESS'] == 'NO') & (df_conectividad_por_localidad['SATELITAL'] == 'NO')
df_conectividad_por_localidad['CONECTIVIDAD'] = 'SI'
df_conectividad_por_localidad.loc[sin_conectividad, 'CONECTIVIDAD'] = 'NO'
df_conectividad_por_localidad['CONECTIVIDAD'].value_counts()

Para realizar un análisis del comportamiento en telecomunicaciones a nivel nacional vamos a primero visualizar el porcentaje de conexión a internet por provincia

In [ ]:
df_conectividad_por_localidad.head(3)

In [ ]:
df_conectividad_por_localidad.to_csv('_src/outputData/conectividad_por_localidad.csv', index=False)

Primero creamos un df con cada provincia unica

In [ ]:
prov_data = pd.DataFrame(df_conectividad_por_localidad['Provincia'].unique())
prov_data.rename({0:'provincias'}, axis=1, inplace=True)

Para cada provincia de ese Dataframe realizamos un conteo de las localidades que cuentan con conectividad a Internet y creamos un campo que explique el porcentaje de conectividad a nivel provincial

In [ ]:
for provincia in prov_data['provincias']:
    prov_data.loc[prov_data['provincias'] == provincia,'loc_conectadas'] = df_conectividad_por_localidad.loc[(df_conectividad_por_localidad['Provincia'] == provincia) & (df_conectividad_por_localidad['CONECTIVIDAD'] == 'SI'),'CONECTIVIDAD'].count()
    prov_data.loc[prov_data['provincias'] == provincia,'loc_sin_conexion'] = df_conectividad_por_localidad.loc[(df_conectividad_por_localidad['Provincia'] == provincia) & (df_conectividad_por_localidad['CONECTIVIDAD'] == 'NO'),'CONECTIVIDAD'].count()
prov_data['porcentaje_conexion'] = round((prov_data['loc_conectadas'] / (prov_data['loc_conectadas'] + prov_data['loc_sin_conexion'])),2) 

prov_data['loc_conectadas'] = prov_data['loc_conectadas'].astype(int)
prov_data['loc_sin_conexion'] = prov_data['loc_sin_conexion'].astype(int)
prov_data['porcentaje_conexion'] = prov_data['porcentaje_conexion'].astype(float)


In [ ]:
sns.set()
plt.scatter(prov_data['porcentaje_conexion'], prov_data['provincias'])
plt.xlabel('porcentaje de conectividad')
plt.ylabel('provincias');

Hay provincias en las cuales la conexion a internet es casi o completa como en CABA, SANTA CRUZ, CORRIENTES, SANTA FE.
Y otras en las que no cuentan con mucha conectividad como CATAMARCA, LA RIOJA, SAN LUIS. Esto puede deberse a distintos factores, pero antes de sacar conclusiones sigamos analizando un poco mas a detalle

Veamos como se comportan las conexiones por tecnología a nivel provincial

In [ ]:
conexiones = ['ADSL','CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', '4G', '3G', 'TELEFONIAFIJA', 'WIRELESS', 'SATELITAL']
for provincia in prov_data['provincias']:
    for conexion in conexiones:
        prov_data.loc[prov_data['provincias'] == provincia, conexion] = df_conectividad_por_localidad.loc[(df_conectividad_por_localidad['Provincia'] == provincia) & (df_conectividad_por_localidad[conexion] == 'SI')].count()[0]

for conexion in conexiones:
    prov_data[conexion] = round(prov_data[conexion] / prov_data['loc_conectadas'], 2)

In [ ]:
prov_data.head(3)

Ahora basandonos en el gráfico anteriormente realizado, busquemos aquellas provincias cuyo umbral de conexión sea menor a 0.77

In [ ]:
prov_data.loc[prov_data['porcentaje_conexion'] < 0.77]

Conclusiones:

#### `Misiones`:
En Misiones, el porcentaje de conexión es del `75%`, lo que indica una conectividad ligeramente debajo del promedio en general.
Los tipos de ``conexiones predominantes`` en Misiones son ``4G, TELEFONIAFIJA y WIRELESS``, con valores de 0.80, 0.55 y 0.46 respectivamente.
Estos datos sugieren que ``la conectividad en Misiones se encuentra en su mayoria en tecnologías inalámbricas y telefónicas.``. 

#### `Jujuy`:
En Jujuy, el porcentaje de conexión es del `61%`, lo que indica una baja conectividad en general.
Los tipos de ``conexiones predominantes`` en Jujuy ``son FIBRAOPTICA, 4G y SATELITAL``, con valores de 0.45, 0.46 y 0.45 respectivamente.
Estos datos sugieren que ``la conectividad en Jujuy es variada pero faltan llegar a algunas localidades aún.``. 

### ``San Luis``:
San Luis tiene un porcentaje de conexión del 46%, lo que indica una baja conectividad en comparación con otras provincias.
El tipo de ``conexión predominante en San Luis es WIRELESS, con un valor de 0.84``.
Otros tipos de conexiones presentes en San Luis, aunque en menor medida, son ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, 4G, 3G, TELEFONIAFIJA y SATELITAL, con valores que van desde 0.10 hasta 0.39.
``Estos datos sugieren que en San Luis, la conectividad está principalmente basada en conexiones inalámbricas (WIRELESS)``. Esto indica que la infraestructura de conectividad en la provincia puede depender en gran medida de tecnologías inalámbricas para proporcionar acceso a internet.

### ``Tierra del Fuego``:

Tierra del Fuego tiene un porcentaje de conexión del 38%, lo que indica una baja conectividad en comparación con otras provincias.
``El tipo de conexión predominante en Tierra del Fuego es 4G, con un valor de 1.00``.
Otros tipos de conexiones presentes en Tierra del Fuego, aunque en menor medida, son ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, 3G, TELEFONIAFIJA y SATELITAL, con valores que van desde 0.33 hasta 0.50.
Estos datos sugieren que en Tierra del Fuego, ``la conectividad está principalmente basada en conexiones móviles de 4G``. Esto indica que la infraestructura de conectividad terrestre en la provincia puede estar enfocada en proporcionar conectividad a través de redes móviles de alta velocidad.

### ``La Rioja``:

En La Rioja, el porcentaje de conexión es del 71%, lo que indica una conectividad relativamente mejor en comparación con las provincias anteriores.
El tipo de conexión predominante en La Rioja es WIRELESS, con un valor de 0.80.
Otros tipos de conexiones presentes en La Rioja, aunque en menor medida, son ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, 4G, 3G, TELEFONIAFIJA y SATELITAL, con valores que van desde 0.20 hasta 0.55.
Estos datos sugieren que en La Rioja, la conectividad está basada principalmente en conexiones inalámbricas (WIRELESS).

### ``Catamarca``:
Catamarca tiene un porcentaje de conexión del 51%, lo que indica una conectividad moderada en comparación con las otras provincias mencionadas.
Los tipos de conexiones más predominantes en Catamarca son 4G, SATELITAL, con valores de 0.61 y 0.52 respectivamente.
Estos datos sugieren que en Catamarca, la conectividad se basa en conexiones 4G y SATELITAL.

Un factor que puede ser indicativo de la baja conectividad a nivel provincial puede ser dada debido a la falta de disponibilidad de redes alternativas a las telefónicas o inalambricas, así como también la falta de infraestructura para la implementación de las mismas.


# Provincias con menor alcance de Fibra Óptica

Siendo la Fibra Óptica el mejor medio actualmente para la transmisión de información veamos cuales son aquellas provincias con menor disposición de esta tecnología a ver si podemos ver alguna relacion o información interesante

In [ ]:
prov_data.sort_values('FIBRAOPTICA', ascending=True).head(5)

Podemos observar que en aquellas provincias cuya conectivad de Fibra Óptica es baja, tiende a prevalecer las conexiones telefónicas, como el 4g, o tecnologías inalámbricas: Satelital y Wireless

# Ingresos por la prestacion del servicio de internet

Realizemos un análisis de los ingresos por la prestacion del servicio de internet

In [ ]:
ingresos_trim = pd.read_csv('_src/data/Internet_Ingresos_trimestrales.csv')
ingresos_trim.head(10)

A simple vista podemos notar que a medida que aumenta el año tambien los ingresos, esto sugiere a que puede haber una relación lineal entre estas variables. Debido a esto realizaremos un modelo que nos ayude a estimar el ingreso del proximo trimestre

In [ ]:
ingresos_trim.dtypes

Cambiamos el formato y tipo de los datos para realizar nuestro análisis

In [ ]:
ingresos_trim['Ingresos (miles de pesos)'] = ingresos_trim['Ingresos (miles de pesos)'].str.replace('.','')
ingresos_trim['Ingresos (miles de pesos)'] = ingresos_trim['Ingresos (miles de pesos)'].astype(int)

Creamos la columna index la cual nos ayudará con el modelo al darle una relación mas lineal a los datos

In [ ]:
ingresos_trim['index'] = sorted(ingresos_trim.index, reverse=True)

In [ ]:
ingresos_trim.head(5)

Creamos nuestro dataframe que utilizaremos para el modelo

In [ ]:
model_data = ingresos_trim[['Ingresos (miles de pesos)','index']].copy()

In [ ]:
mapa = model_data.corr()
mapa.style.background_gradient(cmap='coolwarm')

Importamos las librerías necesarias para realizar el modelo

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

Definimos las variables independientes y la objetivo

In [ ]:
x2 = model_data['index'].values
X = x2.reshape(-1,1)
y = model_data['Ingresos (miles de pesos)'].values

Transformamos la variable independiente para que está sea adecuada para la regresión polinómica

In [ ]:
poly = PolynomialFeatures(8, include_bias=False)

Xpol = poly.fit_transform(X)

In [ ]:
# Instanciamos un nuevo modelo de regresión lineal, pero ahora tendrá los atributos polinómicos que definimos en la celda anterior
reg_2 = LinearRegression()

# Entrenamos la regresión con los atributos X polinómicos 
reg_2.fit(Xpol, y)

y_pred = reg_2.predict(Xpol)

Notemos que el modelo fue entrenado con la totalidad de los datos, esto se debe a que debido a la poca cantidad de datos que tenemos, y que queremos predecir lo mas exactamente los ingresos proximos con los datos que contamos

In [ ]:
plt.figure(figsize = (40,8))
plt.subplot(1,3,1)
plt.scatter(X,y, s = 45, label = 'Datos')

list1, list2 = zip(*sorted(zip(Xpol[:,0], y_pred)))
plt.plot(list1, list2, '--', label ='Regresión', color='black')
plt.legend();

Realizamos la predicción del proximo trimestre en función a nuestro modelo

In [ ]:
proximo_trim = poly.fit_transform([[35]])
proximo_trim = (reg_2.predict(proximo_trim))

In [ ]:
proximo_trim[0]

Se estima un ingreso de 75.091.660 para el 4to trimestre del 2022.

In [ ]:
prov_data.to_csv('_src/outputData/porcentaje_conexion.csv',index=False)

Exportamos nuestra predicción para utilizarla en el dashboard

In [ ]:
prediccion = {'valor': [int(proximo_trim[0])]}
prediccion_ing_prox_trim = pd.DataFrame(prediccion)

In [ ]:
prediccion_ing_prox_trim.to_csv('_src/outputData/prediccion.csv', index=False)

Con los datos del análisis y teniendo un panorama mas general del mercado ondaremos mas profundamente el análisis en conjunto con el dashboard.